In [1]:
!pip install ipython-autotime
%load_ext autotime
import pandas as pd
from google.colab import drive
import pathlib
import numpy as np
import gc
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import GRU
from keras.layers.core import Dropout,Dense
from keras.callbacks import ModelCheckpoint
import pyarrow.parquet as pq
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


drive.mount('/content/drive',force_remount=True)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 4.7 MB/s 
Mounted at /content/drive
time: 19.8 s (started: 2022-11-26 06:13:38 +00:00)


In [2]:
data_dir = "/content/drive/MyDrive/yellow_tripdata_2020_data.parquet"
data_dir = pathlib.Path(data_dir)
yellowTaxiData2020 = pq.ParquetFile(data_dir)
del data_dir

time: 195 ms (started: 2022-11-26 06:13:58 +00:00)


In [3]:
random_state=17

time: 535 µs (started: 2022-11-26 06:13:58 +00:00)


In [4]:
taxiData = pd.DataFrame()

time: 1.7 ms (started: 2022-11-26 06:13:58 +00:00)


In [5]:
for taxiDataBatch in yellowTaxiData2020.iter_batches(batch_size=2000000):
    taxiDataBatchDF = taxiDataBatch.to_pandas()
    taxiDataBatchDF.drop(columns=['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'pickup_date', 'pickup_time', 'dropoff_date', 'dropoff_time'], inplace=True)
    taxiData = pd.concat([taxiData, taxiDataBatchDF])

time: 2.86 s (started: 2022-11-26 06:13:58 +00:00)


In [6]:
del yellowTaxiData2020
gc.collect()

44

time: 161 ms (started: 2022-11-26 06:14:01 +00:00)


In [7]:
data_dir = "/content/drive/MyDrive/yellow_tripdata_2021_data.parquet"
data_dir = pathlib.Path(data_dir)
yellowTaxiData2021 = pq.ParquetFile(data_dir)
del data_dir

time: 134 ms (started: 2022-11-26 06:14:01 +00:00)


In [8]:
for taxiDataBatch in yellowTaxiData2021.iter_batches(batch_size=2000000):
    taxiDataBatchDF = taxiDataBatch.to_pandas()
    taxiDataBatchDF.drop(columns=['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'pickup_date', 'pickup_time', 'dropoff_date', 'dropoff_time'], inplace=True)
    taxiData = pd.concat([taxiData, taxiDataBatchDF])

time: 15.9 s (started: 2022-11-26 06:14:01 +00:00)


In [9]:
def createGRUDataSet(data, sequence_length, feature_count): 
    X = []
    y = []

    for i in range(data.shape[0] - feature_count - 1):
        if data[i, 0] == data[i+1, 0]:
            if data[i, 0] == data[i+sequence_length, 0]:
                X.append(data[i:i+sequence_length, :feature_count])
                y.append(data[i+sequence_length, feature_count])
    
    return np.array(X), np.array(y)

time: 1.53 ms (started: 2022-11-26 06:14:17 +00:00)


In [10]:
taxiDataGRU = taxiData[['PULocationID', 'DOLocationID', 'pickup_weekday', 'pickup_month', 'pickup_dt', 'trip_distance', 'pickup_holiday', 'pickup_near_holiday', 'ride_duration']]

time: 2.24 s (started: 2022-11-26 06:14:17 +00:00)


In [11]:
taxiDataGRU.sort_values(by=['PULocationID', 'DOLocationID', 'pickup_weekday', 'pickup_month', 'pickup_dt'], inplace=True, ascending=True)

time: 35.9 s (started: 2022-11-26 06:14:19 +00:00)


/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [12]:
del taxiData
gc.collect()

88

time: 140 ms (started: 2022-11-26 06:14:55 +00:00)


In [13]:
scaler = MinMaxScaler()
taxiDataGRU = scaler.fit_transform(taxiDataGRU)

time: 2.39 s (started: 2022-11-26 06:14:55 +00:00)


In [14]:
X, y = createGRUDataSet(taxiDataGRU, 5, 8)

time: 1min 1s (started: 2022-11-26 06:14:58 +00:00)


In [15]:
del yellowTaxiData2021, taxiDataBatchDF
gc.collect()

66

time: 141 ms (started: 2022-11-26 06:15:59 +00:00)


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.4, 
                                                    random_state=random_state)

time: 6.01 s (started: 2022-11-26 06:16:00 +00:00)


In [17]:
np.random.seed(random_state)

time: 713 µs (started: 2022-11-26 06:16:06 +00:00)


In [18]:
del X, y
gc.collect()

44

time: 159 ms (started: 2022-11-26 06:16:06 +00:00)


In [19]:
model = Sequential()
model.add(GRU(units=128, input_shape=(5,8), return_sequences=True))
model.add(Dropout(0.2, seed=random_state))
model.add(GRU(units=128, return_sequences=False))
model.add(Dropout(0.2, seed=random_state))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='rmsprop')

time: 848 ms (started: 2022-11-26 06:16:06 +00:00)


In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 5, 128)            52992     
                                                                 
 dropout (Dropout)           (None, 5, 128)            0         
                                                                 
 gru_1 (GRU)                 (None, 128)               99072     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 152,193
Trainable params: 152,193
Non-trainable params: 0
_________________________________________________________________
time: 27.2 ms (started: 2022-11-26 06:16:07 +00:00

In [21]:
gc.collect()

2710

time: 145 ms (started: 2022-11-26 06:16:07 +00:00)


In [22]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', min_delta=0.01, mode='min')

time: 63.7 ms (started: 2022-11-26 06:16:07 +00:00)


In [23]:
model.fit(X_train, y_train, epochs=5, batch_size=714,shuffle=True, callbacks=[callback])

Epoch 1/5
21492/21492 [==============================] - 1031s 48ms/step - loss: 0.0379
Epoch 2/5
21492/21492 [==============================] - 1059s 49ms/step - loss: 0.0357


time: 34min 50s (started: 2022-11-26 06:16:07 +00:00)


In [24]:
model.save('/content/drive/MyDrive/Colab Notebooks/gru_ny_model')

time: 8.87 s (started: 2022-11-26 06:50:58 +00:00)


In [25]:
predictions = model.predict(X_test)

319693/319693 [==============================] - 1217s 4ms/step
time: 21min 59s (started: 2022-11-26 06:51:06 +00:00)


In [26]:
predictions_train = model.predict(X_train)

479539/479539 [==============================] - 1936s 4ms/step
time: 34min 54s (started: 2022-11-26 07:13:06 +00:00)


In [27]:
rmse_train = mean_squared_error(y_train, predictions_train)**0.5
print(f'The RMSE of training prediction is: {rmse_train}')

The RMSE of training prediction is: 0.18702378063879022
time: 571 ms (started: 2022-11-26 07:48:02 +00:00)


In [28]:
mse_train = mean_squared_error(y_train, predictions_train)
print(f'The MSE of training prediction is: {mse_train}')

The MSE of training prediction is: 0.03497789452442632
time: 103 ms (started: 2022-11-26 07:48:02 +00:00)


In [29]:
mae_train = mean_absolute_error(y_train, predictions_train)
print(f'The MAE of training prediction is: {mae_train}')

The MAE of training prediction is: 0.15051484673068372
time: 355 ms (started: 2022-11-26 07:48:02 +00:00)


In [30]:
rmse_test = mean_squared_error(y_test, predictions)**0.5
print(f'The RMSE of prediction is: {rmse_test}')

The RMSE of prediction is: 0.18712379721134817
time: 255 ms (started: 2022-11-26 07:48:03 +00:00)


In [31]:
mse_test = mean_squared_error(y_test, predictions)
print(f'The MSE of prediction is: {mse_test}')

The MSE of prediction is: 0.035015315482793755
time: 66.8 ms (started: 2022-11-26 07:48:03 +00:00)


In [32]:
mae_test = mean_absolute_error(y_test, predictions)
print(f'The MAE of training prediction is: {mae_test}')

The MAE of training prediction is: 0.15062226691178393
time: 77.2 ms (started: 2022-11-26 07:48:03 +00:00)
